<a href="https://colab.research.google.com/github/Venture-Coding/SUNY-Buffalo-ML-and-self-learning/blob/main/NLP/BERT_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## POC for FAQ Bot 


In [ ]:
pip install transformers

     |████████████████████████████████| 3.1 MB 10.2 MB/s 
     |████████████████████████████████| 61 kB 504 kB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
     |████████████████████████████████| 895 kB 62.5 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install tensorflow

In [ ]:

from transformers import BertForQuestionAnswering, AutoTokenizer

modelname = 'deepset/bert-base-cased-squad2'


Deepset is the AI org.  
bert is the model 
base is the base model , other is large.  
cased differentiation exists here, not in uncased model.  
squad2 is the corpus qna on which they trained the model.

In [ ]:

model = BertForQuestionAnswering.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

BERT has quite a few special tokens.

| Token | Meaning | Token ID |
| --- | --- | --- |
| **[PAD]** | Padding token, allows us to maintain same-length sequences (512 tokens for Bert) even when different sized sentences are fed in | 0 |
| **[UNK]** | Used when a word is unknown to Bert | 100 |
| **[CLS]** | Appears at the start of every sequence | 101 |
| **[SEP]** | Indicates a seperator - used to indicate point between context-question and appears at end of sequences | 102 |
| **[MASK]** | Used when masking tokens, for example in training with masked language modelling (MLM) | 103 |

In [ ]:
questions = [
             "How quickly do you add new browser versions to BrowserStack?",
             "Which browsers and OSes do you provide for cross-browser testing on Live?",
             "Can I start simultaneous BrowserStack Live sessions using one account?"
]

In [ ]:
d = tokenizer.encode(questions[0] , truncation= True, padding= True)
print(d,)

[101, 1409, 170, 1207, 19089, 1683, 1110, 1308, 117, 1165, 1110, 1122, 1896, 1106, 139, 20876, 1200, 1708, 1777, 2158, 136, 102]


BERT ideally wants 512. So padding will take a lesser count to 512 and truncation will bring it down to that number shedding excess.

so how it works for the question-answering model is  
[CLS] .context. [SEP] .?. [PAD][PAD].... upto 512

In [ ]:
context = "We usually add a new browser version within a week of its stable release. Wherever possible, we also provide access to the beta version of a browser. We provide browsers running on different versions of Windows, Mac OS X, Android and iOS in Live. See the complete list of browsers, OSes and mobile devices on Browsers and Platforms. Each user can only have one active BrowserStack Live session at a time. For multi-user plans, each user needs to have an individual account, even if they are part of the same Organization. Therefore, depending on the user limit, there can be multiple BrowserStack Live sessions per Organization, but only one per account."

In [ ]:
from transformers import pipeline
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [ ]:
nlp({
    'question': questions[2],
     'context': context
})

{'answer': 'depending on the user limit',
 'end': 560,
 'score': 0.12949934601783752,
 'start': 533}

In [ ]:
nlp({
    'question': questions[1],
     'context': context
})

{'answer': 'different versions of Windows, Mac OS X, Android and iOS',
 'end': 237,
 'score': 0.0124394865706563,
 'start': 181}

In [ ]:
nlp({
    'question': questions[0],
     'context': context
})

{'answer': 'within a week',
 'end': 50,
 'score': 0.6914547681808472,
 'start': 37}

Makes sense since the Stanford QnA Dataset corpus has accounted for Syntactic variance as well as Lexical variance(both synonymy and syntactic dependency), makes it easier for us to put paragraphs and get paraphrased answers with high confidence/relevance.